In [1]:
import pandas as pd
from pyvi import ViUtils
import numpy as np

In [2]:
tiki_df = pd.read_csv('../data/tiki.csv')
for column in tiki_df.columns:
    print(column)

Tên sản phẩm
Gi á
Bluetooth
Thương hiệu
Xuất xứ
Thẻ nhớ ngoài
Chipset
tốc độ CPU
Kích thước 
Công nghệ màn hình
Jack tai nghe
Loại pin
Loại sim
Trọng lượng 
RAM
Độ phân giải
ROM
Kích thước màn hình


In [3]:
drop_columns = ['Thương hiệu', 'Xuất xứ', 'Thẻ nhớ ngoài', 'tốc độ CPU', 'Kích thước ', 'Jack tai nghe', 'Trọng lượng ', 'Độ phân giải', 'Bluetooth']

for column in drop_columns:
    del tiki_df[column]

tiki_df.head()

,Tên sản phẩm,Gi á,Chipset,Công nghệ màn hình,Loại pin,Loại sim,RAM,ROM,Kích thước màn hình
0,Điện Thoại Oppo A76 (6GB/128G) - Hàng Chính Hãng,5.490.000 ₫,Snapdragon 680 8 nhân,IPS LCD,NaN,2 nano sim,6 GB,128 GB,6.56 inch
1,Điện thoại Smartphone Sugar A100 2GB/16GB 4G L...,1.159.000 ₫,Mediatek MT6739 4 nhân 4x1.5GHz,NaN,NaN,NaN,NaN,NaN,NaN
2,Điện thoại Samsung Galaxy A32 (6GB/128GB) - Hà...,5.030.000 ₫,MediaTek Helio G80,Super AMOLED,Li-Ion,Nano SIM,NaN,NaN,6.4 inch
3,Điện thoại Xiaomi 11T Pro (12GB/256GB) - Hàng ...,13.590.000 ₫,Snapdragon 888 8 nhân,AMOLED,Li-Po,Nano SIM,12GB,256GB,6.67 inch
4,Điện Thoại Xiaomi Redmi 9C 4GB/128GB - Hàng Ch...,3.059.000 ₫,MediaTek Helio G35 8 nhân,IPS LCD,Li-Ion,Nano,4GB,128GB,6.53 inch


# Xử lý tên cột

In [4]:
tiki_df = tiki_df.rename(columns={"Tên sản phẩm": "ten", "Gi á": "gia", "Chipset": "cpu", "Loại pin": "pin", "Loại sim": "sim", "ROM": "bộ nhớ trong"})

for column in tiki_df.columns:
    tiki_df = tiki_df.rename(columns={column: ViUtils.remove_accents(column).decode("utf-8").lower().strip().replace(' ', '_')})

# Xử lý trường tên

In [5]:
tiki_df = tiki_df[tiki_df['ten'].notna()]
tiki_df['ten'] = tiki_df['ten'].apply(lambda s: str(s).split('-')[0].replace('Điện thoại', '').replace('Điện Thoại', '').strip())
tiki_df['ten'] = tiki_df['ten'].apply(lambda s: ViUtils.remove_accents(s).decode("utf-8"))

# Xử lý trường giá

In [6]:
tiki_df = tiki_df[tiki_df['gia'].notna()]
tiki_df['gia'] = tiki_df['gia'].apply(lambda s: str(s).replace('₫', '').replace('.', '').replace('*', '').strip())
tiki_df['gia'] = tiki_df['gia'].apply(lambda s: int(s))

# Xử lý trường kích thước màn hình

In [7]:
tiki_df = tiki_df[tiki_df['kich_thuoc_man_hinh'].notna()]
def process_screen_size(s):
    size = s.split(' - ')[0]
    try:
        if "&" in s:
            size = size.split(' & ')
            size1 = float(size[0].split()[1])
            size2 = float(size[1].split()[1])
            size = size1 + size2
        else:
            size = float(size.split()[0])
    except:
        size = 0
    return size

tiki_df['kich_thuoc_man_hinh'] = tiki_df['kich_thuoc_man_hinh'].apply(process_screen_size)

# Xử lý trường bộ nhớ trong

In [8]:
def get_rom(s):
    index = s.find('/')
    if index == -1:
        return "not found"
    inf =  s[index:]
    ind = inf.find('GB')
    inf = inf[1:ind+2]
    return inf

tiki_df = tiki_df.reset_index()  # make sure indexes pair with number of rows

for index, row in tiki_df.iterrows():
    if type(row['bo_nho_trong']) != str:
        tiki_df.loc[index, 'bo_nho_trong'] = get_rom(row['ten'])

In [9]:
def process_rom(s):
    try:
        return float(s[:-2].strip())
    except:
        return 0

tiki_df['bo_nho_trong'] = tiki_df['bo_nho_trong'].apply(process_rom)

# Xử lý trường RAM

In [10]:
def get_ram(s):
    index = s.find('/')
    if index == -1:
        return "not found"
    inf = s[:index]
    inf = inf[index-3:index]
    return inf

tiki_df = tiki_df.reset_index()  # make sure indexes pair with number of rows

for index, row in tiki_df.iterrows():
    if type(row['ram']) != str:
        tiki_df.loc[index, 'ram'] = get_ram(row['ten'])

In [11]:
def process_ram(s):
    try:
        return int(s[:-2])
    except:
        return 0

tiki_df['ram'] = tiki_df['ram'].apply(process_ram)

In [12]:
del tiki_df['level_0']
del tiki_df['index']

In [13]:
for column in tiki_df.columns:
    print(column)

ten
gia
cpu
cong_nghe_man_hinh
pin
sim
ram
bo_nho_trong
kich_thuoc_man_hinh


In [14]:
tiki_df.to_csv('../clean_data/tiki.csv', index=False)